<a href="https://colab.research.google.com/github/eispoohw/CS493-Math-Methods-in-ML/blob/main/lab%204.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Деревья решений

## Построение дерева

Опишем жадный алгоритм построения бинарного дерева решений:
1. Начинаем со всей обучающей выборки $X$, которую помещаем в корень $R_1$. 
2. Задаём функционал качества $Q(X, j, t)$ и критерий остановки. 
3. Запускаем построение из корня: $SplitNode(1, R_1)$

Функция $SplitNode(m, R_m)$
1. Если выполнен критерий остановки, то выход.
2. Находим наилучший с точки зрения $Q$ предикат: $j, t$: $[x_j<t]$
3. Помещаем предикат в вкршину и получаем с его помощью разбиение $X$ на две части: $R_{left} = \lbrace x|x_j<t \rbrace$ и $R_{right} = \lbrace x|x_j \geqslant t \rbrace$
4. Поместим $R_{left}$ и $R_{right}$ соответсвенно в левое и правое поддерево.
5. Рекурсивно повторяем $SplitNode(left, R_{left})$ и $SplitNode(right, R_{right})$.

В конце поставим в соответствие каждому листу ответ. Для задачи классификации - это самый частый среди объектов класс или вектор с долями классов (можно интерпретировать как вероятности):
$$ c_v = \arg \max_{k\in Y} \sum_{(x_i,y_i) \in R_v} [y_i=k]  $$

## Функционал качества для деревьев решений


Энтропия Шеннона для системы с N возможными состояниями определяется по формуле:
$$H = - \sum_{i=0}^{N} p_i\log_2p_i $$

где $p_i$ – вероятности нахождения системы в $i$-ом состоянии. 

Это очень важное понятие теории информации, которое позволяет оценить количество информации (степень хаоса в системе). Чем выше энтропия, тем менее упорядочена система и наоборот. С помощью энтропии мы формализуем функционал качества для разделение выборки (для задачи классификации).

In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

import random
from pprint import pprint

Код для расчёта энтропии:

In [2]:
def entropy(y):
    
    _, counts = np.unique(y, return_counts=True)

    probabilities = counts / counts.sum()
    entropy = sum(probabilities * -np.log2(probabilities))
     
    return entropy

Здесь $y$ - это массив значений целевой переменной

Энтропия – по сути степень хаоса (или неопределенности) в системе. Уменьшение энтропии называют приростом информации (information gain, IG).

Обочначим $R_v$ - объекты, которые нужно разделить в помощью предиката в вершине $v$. Запишем формулу для расчёта информационного прироста:
$$ Q = IG = H(R_v) - (H(R_{left})+H(R_{right}))$$

На каждом шаге нам нужно максимизировать этот функционал качества. Как это делать? Например, так можно перебрать $t$ для выбранного $j$.

Предыдущая версия формулы прироста информации слишком упрощена. В работе необходимо использовать более устойчивую формулу, которая учитывает не только энтропию подмножеств, но и их размер. 

$$ Q = IG = H(R_v) - \Big (\frac{|R_{left}|} {|R_{v}|} H(R_{left})+ \frac{|R_{right}|} {|R_{v}|} H(R_{right})\Big)$$

где, $|R_{v}|$, $|R_{left}|$ и $|R_{right}|$ - количество элементов в соответствующих множествах.


### Задание 4.1

Реализуйте алгоритм построения дерева. Должны быть отдельные функции (методы) для расчёта энтропии (уже есть), для разделения дерева (используйте `pandas`), для подсчёта функционала качества $IG$, для выбора наилучшего разделения (с учетом признакоd и порогов), для проверки критерия остановки.

Для набора данных `iris` реализуйте алгоритм и минимум три из разными критерия остановки из перечисленных ниже:
* максимальной глубины дерева = 5
* минимального числа объектов в листе = 5
* максимальное количество листьев в дереве = 5
* purity (остановка, если все объекты в листе относятся к одному классу)

Реализуйте функцию `predict` (на вход функции подаётся датафрейм с объектами)

Оцените точность каждой модели с помощью метрики доля правильных ответов (`from sklearn.metrics import accuracy_score` или реализовать свою).

In [3]:
from sklearn.metrics import accuracy_score
from collections import Counter

class TreeNode:
    def __init__(self):
        self.predicateBound: float = None # dividing value
        self.predicateName: str = None # feature in predicate
        self.left: TreeNode = None 
        self.right: TreeNode = None
        self.val = None # if Node is a leaf


class DecisionTree:
    def __init__(self, depth=None, objectsInLeaf=None, purity=True):
        if not objectsInLeaf:
            objectsInLeaf = 1
        self.depth = depth    
        self.objectsInLeaf = objectsInLeaf
        self.purity = purity

    def fit(self, X, y) -> TreeNode:
        self.features = X.columns
        if not self.depth:
            self.depth = len(y)
        self.root = self.splitNode(X, y, 1)
        return self.root

    def splitNode(self, X, y, depth) -> TreeNode:
        node = TreeNode()
        if depth == self.depth or self.purity and self.entropy(y) == 0:
            node.val = self.getNodeVal(y)
            return node
        df = X.copy()
        df['target'] = y
        maxIG, dfR, dfL = 0, None, None
        # searching max IG through each unique value of each feature 
        for f in self.features:
            for value in set(X[f]):
                dfRight = df[df[f] >= value]
                dfLeft = df[df[f] < value]
                # min objects in leaf criteria
                if len(dfRight) < self.objectsInLeaf or len(dfLeft) < self.objectsInLeaf:
                    continue
                IG = self.IG(y, dfRight['target'], dfLeft['target'])
                if IG > maxIG:
                    maxIG = IG
                    node.predicateName = f
                    node.predicateBound = value
                    dfR, dfL = dfRight, dfLeft
        if maxIG == 0:
            node.val = self.getNodeVal(y)
            return node
        node.left = self.splitNode(dfL.drop(['target'], axis=1), dfL['target'], depth + 1)
        node.right = self.splitNode(dfR.drop(['target'], axis=1), dfR['target'], depth + 1)
        return node

    def predict(self, X):
        res = np.array([])
        for i in range(len(X)):
            res = np.append(res, self.checkNode(self.root, X.iloc[[i]]))
        return res

    def checkNode(self, node, x):
        if node.left is None and node.right is None:
            return node.val
        if x[node.predicateName].iloc[0] < node.predicateBound:
            return self.checkNode(node.left, x)
        return self.checkNode(node.right, x)

    def getNodeVal(self, y):
        classCounts = Counter(y)
        res = ""
        maxval = 0
        for c in classCounts:
            if classCounts[c] > maxval:
                maxval = classCounts[c]
                res = c
        return res

    def entropy(self, y):
        return entropy(y)

    def IG(self, prev, left, right):
        return self.entropy(prev) - (len(left)/len(prev)*self.entropy(left) + len(right)/len(prev)*self.entropy(right))

    def accuracy(self, predict, real):
        return accuracy_score(real, predict)

    def showTree(self, node, parentdepth=0):
        if parentdepth == 0:
            print("DecisionTree: ")
            print(f"- max-depth: {self.depth}\n- min-objects-in-leaf: {self.objectsInLeaf}\n- purity: {self.purity}\n")
        startChars = "|  " * parentdepth + "|__"
        if node.left is None and node.right is None:
            print(startChars + f"class: {node.val}")
        else:
            print(startChars +  node.predicateName + " <", node.predicateBound)
            self.showTree(node.left, parentdepth + 1)
            self.showTree(node.right, parentdepth + 1)


In [4]:
from sklearn import datasets
from sklearn.model_selection import train_test_split, cross_val_score

iris = datasets.load_iris()

X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, random_state=239, test_size=0.33)
X_train = pd.DataFrame(X_train, columns=iris['feature_names'])
X_test = pd.DataFrame(X_test, columns=iris['feature_names'])

Рассмотрим деревья с различными критериями остановки и их accuracy

In [5]:
dt = DecisionTree(depth=5, objectsInLeaf=5, purity=True)

dt.fit(X_train, y_train)
dt.showTree(dt.root)

print("\nAccuracy: ", end=" ")
dt.accuracy(dt.predict(X_test), y_test)

DecisionTree: 
- max-depth: 5
- min-objects-in-leaf: 5
- purity: True

|__petal length (cm) < 3.0
|  |__class: 0
|  |__petal length (cm) < 5.1
|  |  |__petal width (cm) < 1.7
|  |  |  |__class: 1
|  |  |  |__class: 2
|  |  |__class: 2

Accuracy:  

0.96

In [6]:
dt = DecisionTree(purity=True)

dt.fit(X_train, y_train)
dt.showTree(dt.root)

print("\nAccuracy: ", end=" ")
dt.accuracy(dt.predict(X_test), y_test)

DecisionTree: 
- max-depth: 100
- min-objects-in-leaf: 1
- purity: True

|__petal length (cm) < 3.0
|  |__class: 0
|  |__petal length (cm) < 5.1
|  |  |__petal width (cm) < 1.7
|  |  |  |__class: 1
|  |  |  |__sepal width (cm) < 3.0
|  |  |  |  |__class: 2
|  |  |  |  |__sepal length (cm) < 6.7
|  |  |  |  |  |__sepal length (cm) < 6.0
|  |  |  |  |  |  |__class: 1
|  |  |  |  |  |  |__class: 2
|  |  |  |  |  |__class: 1
|  |  |__class: 2

Accuracy:  

0.96

In [7]:
dt = DecisionTree(depth=5, purity=False)

dt.fit(X_train, y_train)
dt.showTree(dt.root)

print("\nAccuracy: ", end=" ")
dt.accuracy(dt.predict(X_test), y_test)

DecisionTree: 
- max-depth: 5
- min-objects-in-leaf: 1
- purity: False

|__petal length (cm) < 3.0
|  |__class: 0
|  |__petal length (cm) < 5.1
|  |  |__petal width (cm) < 1.7
|  |  |  |__class: 1
|  |  |  |__sepal width (cm) < 3.0
|  |  |  |  |__class: 2
|  |  |  |  |__class: 1
|  |  |__class: 2

Accuracy:  

0.94

In [8]:
dt = DecisionTree(objectsInLeaf=5, purity=False)

dt.fit(X_train, y_train)
dt.showTree(dt.root)

print("\nAccuracy: ", end=" ")
dt.accuracy(dt.predict(X_test), y_test)

DecisionTree: 
- max-depth: 100
- min-objects-in-leaf: 5
- purity: False

|__petal length (cm) < 3.0
|  |__class: 0
|  |__petal length (cm) < 5.1
|  |  |__petal width (cm) < 1.7
|  |  |  |__class: 1
|  |  |  |__class: 2
|  |  |__class: 2

Accuracy:  

0.96

##  Случайный лес

Опишем алгоритм случайный лес (*random forest*) и попутно разберём основные идеи:

1. Зададим $N$ - число деревьев в лесу.
2. Для каждого $n$ из $N$ сгенерируем свою выборку $X_n$. Пусть $m$ - это количество объектов в $X$. При генерации каждой $X_n$ мы будем брать объекты $m$ раз с возвращением. То есть один и тот же объект может попасть в выборку несколько раз, а какие-то объекты не попадут. (Этот способ назвается бутстрап).
3. По каждой $X_n$ построим решающее дерево $b_n$. Обычно стараются делать глубокие деревья. В качестве критериев остановки можно использовать `max_depth` или `min_samples_leaf` (например, пока в каждом листе не окажется по одному объекту). При каждом разбиении сначала выбирается $k$ (эвристика $k = \sqrt d$, где $d$ - это число признаков объектов из выборки $X$) случайных признаков из исходных, и оптимальное разделение выборки ищется только среди них. Обратите внимание, что мы не выбрасываем оставшиеся признаки!
4. Итоговый алгоритм будет представлять собой результат голосования (для классификации) и среднее арифметическое (для регрессии). Модификация алгоритма предполагает учёт весов каждого отдельного слабого алгоритма в ансамбле, но в этом особо нет смысла.


### Задание 4.2

В качестве набора данных используйте: https://www.kaggle.com/mathchi/churn-for-bank-customers

Там есть описание и примеры работы с этими данными. Если кратко, речь идёт про задачу прогнозирования оттока клиентов. Есть данные о 10 тысячах клиентов банка, часть из которых больше не являются клиентами.

Используя либо свою реализацию, либо  `DecisionTreeClassifier` с разными настройками из `sklearn.tree` реализйте алгоритм "случайный лес". 

Найдите наилучшие гиперпараметры этого алгоритма: количество деревьев, критерий остановки, функционал качества, минимальное количество объектов в листьях и другие.

Нельзя использовать готовую реализацию случайного леса из `sklearn`.

В подобных задачах очень важна интерпретируемость алгоритма. Попытайтесь оценить информативность признаков, т.е. ответить а вопрос, значения каких признаков являются самыми важными индикаторами того, что банк потеряет клиента.

In [9]:
!gdown 1Gu4armBj-QlwLByFnzWZ85g45GK5eIj6

Downloading...
From: https://drive.google.com/uc?id=1Gu4armBj-QlwLByFnzWZ85g45GK5eIj6
To: /content/churn.csv
100% 685k/685k [00:00<00:00, 89.2MB/s]


In [10]:
from sklearn.preprocessing import OneHotEncoder

df = pd.read_csv('churn.csv')

dfCategorial = df[['Geography']]
df['Gender'] = df['Gender'].apply(lambda x: 0 if x == 'Female' else 1)
df = df.drop(['Geography', 'Surname', 'CustomerId', 'RowNumber'], axis=1)

enc = OneHotEncoder()
X = enc.fit_transform(dfCategorial).toarray()
df = pd.DataFrame(X, columns=enc.get_feature_names_out()).join(df)

X_train, X_test, y_train, y_test = train_test_split(df.drop(['Exited'], axis=1), df['Exited'])
X_train = pd.DataFrame(X_train, columns=df.drop(['Exited'], axis=1).columns)
X_test = pd.DataFrame(X_test, columns=df.drop(['Exited'], axis=1).columns)

df

,Geography_France,Geography_Germany,Geography_Spain,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1.0,0.0,0.0,619,0,42,2,0.00,1,1,1,101348.88,1
1,0.0,0.0,1.0,608,0,41,1,83807.86,1,0,1,112542.58,0
2,1.0,0.0,0.0,502,0,42,8,159660.80,3,1,0,113931.57,1
3,1.0,0.0,0.0,699,0,39,1,0.00,2,0,0,93826.63,0
4,0.0,0.0,1.0,850,0,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,1.0,0.0,0.0,771,1,39,5,0.00,2,1,0,96270.64,0
9996,1.0,0.0,0.0,516,1,35,10,57369.61,1,1,1,101699.77,0
9997,1.0,0.0,0.0,709,0,36,7,0.00,1,0,1,42085.58,1
9998,0.0,1.0,0.0,772,1,42,3,75075.31,2,1,0,92888.52,1


In [11]:
from sklearn.tree import DecisionTreeClassifier

class TwoClassRandomForest:
    def __init__(self, N=100):
        self.N = N
        self.trees = []

    def fit(self, X, y, m=None):
        if not m:
            m = len(y)
        self.m = m

        self.feature_importance = pd.DataFrame()
        self.feature_importance['names'] = X.columns
        self.feature_importance['value'] = 0

        df = X.copy()
        df['target'] = y
        
        for i in range(self.N):
            self.trees.append(DecisionTreeClassifier(max_depth=5, criterion='gini', min_samples_leaf=1, max_features='sqrt'))
            curr_sample = df.sample(n=self.m, replace=True)
            self.trees[-1].fit(curr_sample.drop(['target'], axis=1), curr_sample['target'])
            self.feature_importance['value'] += self.trees[-1].feature_importances_
        self.feature_importance['value'] /= self.N

    def predict(self, X):
        results = pd.DataFrame()
        results['sum'] = np.zeros(len(X.values))
        for r in self.trees:
            results['sum'] += r.predict(X) 
        return results['sum'].apply(lambda x: 0 if x <= self.N // 2 else 1)

In [12]:
rf = TwoClassRandomForest(N=200)
rf.fit(X_train, y_train, m=5000)
y_predict = rf.predict(X_test)
accuracy_score(y_test, y_predict)

0.8456

Значимость признаков

In [13]:
rf.feature_importance.sort_values(by='value', ascending=False)

,names,value
5,Age,0.389757
8,NumOfProducts,0.297691
10,IsActiveMember,0.093137
1,Geography_Germany,0.068411
7,Balance,0.059795
3,CreditScore,0.028028
4,Gender,0.020172
11,EstimatedSalary,0.017166
0,Geography_France,0.009956
6,Tenure,0.008412
